In [3]:
import cv2
from PIL import Image, ImageOps
import argparse
import os
import glob
import matplotlib.pyplot as plt
import multiprocessing as mp
from joblib import Parallel, delayed
import pydicom
import time
import numpy as np
import torch
import random
import pandas as pd
from pydicom.pixel_data_handlers import apply_windowing
import imageio
import time
import shutil

In [2]:
test = pd.read_csv(r"E:\Download\archive\test_df_fold_1.csv")
train = pd.read_csv(r"E:\Download\archive\train_df_fold_1.csv")


In [4]:
test.image_id

0      P325-R-DM-MLO
1        P21-L-DM-CC
2         P3-R-DM-CC
3      P165-R-DM-MLO
4       P298-R-DM-CC
           ...      
71    P99-L-DM-MLO-2
72     P279-R-DM-MLO
73      P124-L-DM-CC
74      P64-L-DM-MLO
75       P84-L-DM-CC
Name: image_id, Length: 76, dtype: object

In [5]:
train[train.image_id.isin(test.image_id)]

,Unnamed: 0,image_id,Path,Type,status
0,0,P325-R-DM-MLO,Data_image/325CDD/MLO/R/MLO_R_P325-R-DM-MLO.png,cdd,1
1,1,P21-L-DM-CC,Data_image/21CDD/CC/L/CC_L_P21-L-DM-CC.png,cdd,0
2,2,P3-R-DM-CC,Data_image/3CDD/CC/R/CC_R_P3-R-DM-CC.png,cdd,1
3,3,P165-R-DM-MLO,Data_image/165CDD/MLO/R/MLO_R_P165-R-DM-MLO.png,cdd,1
4,4,P298-R-DM-CC,Data_image/298CDD/CC/R/CC_R_P298-R-DM-CC.png,cdd,0
...,...,...,...,...,...
73,73,P124-L-DM-CC,Data_image/124CDD/CC/L/CC_L_P124-L-DM-CC.png,cdd,0
74,74,P64-L-DM-MLO,Data_image/64CDD/MLO/L/MLO_L_P64-L-DM-MLO.png,cdd,1
75,75,P84-L-DM-CC,Data_image/84CDD/CC/L/CC_L_P84-L-DM-CC.png,cdd,2
366,366,P311-L-DM-CC,Data_image/311CDD/CC/L/CC_L_P311-L-DM-CC.png,cdd,1


In [4]:
xx = pd.read_csv(r"E:\Download\xx.csv")

In [5]:
xx

,Unnamed: 0,study_id,series_id,image_id,laterality,view_position,height,width,breast_birads,breast_density,finding_categories,finding_birads,xmin,ymin,xmax,ymax,split
0,0,5683854eafabc34f6d854000d2ac6c2d,4ac33111294b83d43537cb8604b0808c,7385e8cf7b29764525c81de4aa1aebe4,L,MLO,3518,2800,BI-RADS 3,DENSITY C,['Mass'],BI-RADS 3,142.899002,2045.170044,417.876007,2265.879883,test
1,1,7c51789da6c462e55bcb95c2a7d437ee,ac4d0771f6d7a7400ab463458f789dbe,1142cdfbfe725d62038252693cf064e5,L,MLO,3518,2800,BI-RADS 4,DENSITY C,"['Nipple Retraction', 'Mass']",BI-RADS 4,632.033997,1727.099976,930.142029,2190.830078,training
2,2,d94fa669c37cc757ea519ca3c291ccb4,198e0af0975db97437f1b7cf6a826e67,d51acb58b7bbec42dc749b5463863c6f,L,MLO,3518,2800,BI-RADS 3,DENSITY C,['Mass'],BI-RADS 3,168.690002,1760.989990,579.971008,2133.629883,training
3,3,57674c2def92287668ed5bc51092fbef,080c6eda9556d5bf5c10e2299b1f6fbc,040cc172596bded4092c44094dc33fb7,L,MLO,3518,2800,BI-RADS 5,DENSITY C,['Architectural Distortion'],BI-RADS 5,446.209991,1667.469971,702.867981,1965.319946,training
4,4,57674c2def92287668ed5bc51092fbef,080c6eda9556d5bf5c10e2299b1f6fbc,040cc172596bded4092c44094dc33fb7,L,MLO,3518,2800,BI-RADS 5,DENSITY C,['Mass'],BI-RADS 4,156.190002,1279.109985,314.239990,1458.550049,training
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5134,5134,32faa475192319343ddf2d39a34efcbe,0c385b366632459c076f2455bf7bccb0,2de40c4e296397b65ae32d97855a3c02,L,MLO,2812,2012,BI-RADS 2,DENSITY C,['No Finding'],NaN,NaN,NaN,NaN,NaN,training
5135,5135,0bfb4e582c69fa4d06d011e637b44fe8,66a2bae2aa8fdf3b875d2d1dec56eaba,5cc4d08c6d8d91172761b30a2e63a8c0,L,MLO,2812,2012,BI-RADS 1,DENSITY D,['No Finding'],NaN,NaN,NaN,NaN,NaN,training
5136,5136,8db1b8ba11d6d804141f1fa4cf91b614,e26615e35922f3b06ca09a742de0ed8b,bf84bfc8d2bda8d0b2c58b1578e7c694,L,MLO,2812,2012,BI-RADS 2,DENSITY C,['No Finding'],NaN,NaN,NaN,NaN,NaN,training
5137,5137,f2093a752e6b44df5990f5fd38c99dd2,2b1b2b8f48abab9819c0b3d091e152ee,9edcc7df8ac157fb2fdbecb9baae9f2f,L,MLO,2812,2012,BI-RADS 1,DENSITY C,['No Finding'],NaN,NaN,NaN,NaN,NaN,training


In [7]:
for i in xx.image_id:
    if os.path.exists(f'E:/BB/{i}.png'):
        shutil.move(f'E:/BB/{i}.png', 'E:/Download/chon/')

In [23]:
classifical = {'NORM': 0, 'B':-1, 'M':1}
data = {}

In [24]:

with open(r"E:\WORKBASE\Project-rsna-breast-cancer-detection\Mias\Info.txt", 'r') as f:
    for line in f.readlines()[1:]:
        # print(line.split(' '))
        ls = line.split(' ')
        for item in ls:
            if item.strip() == "":
                ls.remove(item)
        if len(ls) <= 3:
            if ls[2] == 'NORM':
                data[ls[0]] = classifical[ls[2]]
            else:
                print(ls)
        else:
            # print(ls[3].replace("\n", ""))
            if "\n" in ls[3]:
                ls[3] = ls[3].replace("\n", "")
            try:
                data[ls[0]] = classifical[ls[3]]
            except:
                print(ls)

In [26]:
mias = pd.DataFrame(columns=['image_id', "status"])
mias

,image_id,status


In [27]:
mias['image_id'] = data.keys()
mias

,image_id,status
0,mdb001,NaN
1,mdb002,NaN
2,mdb003,NaN
3,mdb004,NaN
4,mdb005,NaN
...,...,...
317,mdb318,NaN
318,mdb319,NaN
319,mdb320,NaN
320,mdb321,NaN


In [28]:
mias['status'] = mias.image_id.apply(lambda x: data[x])
mias

,image_id,status
0,mdb001,-1
1,mdb002,-1
2,mdb003,0
3,mdb004,0
4,mdb005,-1
...,...,...
317,mdb318,0
318,mdb319,0
319,mdb320,0
320,mdb321,0


In [34]:
for i in mias.index:
    path1 = f'E:/WORKBASE/Project-rsna-breast-cancer-detection/Mias/all-mias/{mias.image_id[i]}.pgm'
    # print(path1)
    img = cv2.imread(path1)
    # print(img)
    cv2.imwrite(f"E:\WORKBASE\Project-rsna-breast-cancer-detection\DATA_STANDARD\Mias\{mias.image_id[i]}.png", img)

In [35]:
mias

,image_id,status
0,mdb001,-1
1,mdb002,-1
2,mdb003,0
3,mdb004,0
4,mdb005,-1
...,...,...
317,mdb318,0
318,mdb319,0
319,mdb320,0
320,mdb321,0


In [36]:
mias.to_csv("E:\WORKBASE\Project-rsna-breast-cancer-detection\DATA_STANDARD\Mias.csv")

In [5]:
mias = pd.read_csv("E:\WORKBASE\Project-rsna-breast-cancer-detection\DATA_STANDARD\Mias.csv", index_col='Unnamed: 0')
mias.head(5)

,image_id,status
0,mdb001,-1
1,mdb002,-1
2,mdb003,0
3,mdb004,0
4,mdb005,-1


In [6]:
d = {-1: 1, 1: 2, 0: 0}
mias.status = mias.status.apply(lambda x: d[x])

In [7]:
mias.status.value_counts()

status
0    207
1     63
2     52
Name: count, dtype: int64